In [ ]:
import random, math, time
from operator import itemgetter
from _functools import reduce
from copy import copy, deepcopy
from builtins import isinstance
from heapq import heappop, heappush
# from resource import setrlimit, RLIMIT_AS, RLIMIT_DATA


class NPuzzle:
    """
    Reprezentarea unei stări a problemei și a istoriei mutărilor care au adus starea aici.

    Conține funcționalitate pentru
    - afișare
    - citirea unei stări dintr-o intrare pe o linie de text
    - obținerea sau ștergerea istoriei de mutări
    - obținerea variantei rezolvate a acestei probleme
    - verificarea dacă o listă de mutări fac ca această stare să devină rezolvată.
    """

    NMOVES = 4
    UP, DOWN, LEFT, RIGHT = range(NMOVES)
    ACTIONS = [UP, DOWN, LEFT, RIGHT]
    names = "UP, DOWN, LEFT, RIGHT".split(", ")
    BLANK = ' '
    delta = dict(zip(ACTIONS, [(-1, 0), (1, 0), (0, -1), (0, 1)]))

    PAD = 2

    def __init__(self, puzzle : list[int], movesList : list[int] = []):
        """
        Creează o stare nouă pe baza unei liste liniare de piese, care se copiază.

        Opțional, se poate copia și lista de mutări dată.
        """
        self.N = len(puzzle)
        self.side = int(math.sqrt(self.N))
        self.r = copy(puzzle)
        self.moves = copy(movesList)

    def display(self, show = True) -> str:
        l = "-" * ((NPuzzle.PAD + 1) * self.side + 1)
        aslist = self.r

        slices = [aslist[ slice * self.side : (slice+1) * self.side ]  for slice in range(self.side)]
        s = ' |\n| '.join([' '.join([str(e).rjust(NPuzzle.PAD, ' ') for e in line]) for line in slices]) 

        s = ' ' + l + '\n| ' + s + ' |\n ' + l
        if show: print(s)
        return s
    
    def display_moves(self):
        print([names[a] if a is not None else None for a in moves])

    def print_line(self):
        return str(self.r)

    @staticmethod
    def read_from_line(line : str):
        list = line.strip('\n][').split(', ')
        numeric = [NPuzzle.BLANK if e == "' '" else int(e) for e in list]
        return NPuzzle(numeric)

    def clear_moves(self):
        """Șterge istoria mutărilor pentru această stare."""
        self.moves.clear()

    def apply_move_inplace(self, move : int):
        """Aplică o mutare, modificând această stare."""
        blankpos = self.r.index(NPuzzle.BLANK)
        y, x = blankpos // self.side, blankpos % self.side
        ny, nx = y + NPuzzle.delta[move][0], x + NPuzzle.delta[move][1]
        if ny < 0 or ny >= self.side or nx < 0 or nx >= self.side: return None
        newpos = ny * self.side + nx
        piece = self.r[newpos]
        self.r[blankpos] = piece
        self.r[newpos] = NPuzzle.BLANK
        self.moves.append(move)
        return self

    def apply_move(self, move : int):
        """Construiește o nouă stare, rezultată în urma aplicării mutării date."""
        return self.clone().apply_move_inplace(move)

    def solved(self):
        """Întoarce varianta rezolvată a unei probleme de aceeași dimensiune."""
        return NPuzzle(list(range(self.N))[1:] + [NPuzzle.BLANK])

    def verify_solved(self, moves : list[int]) -> bool:
        """"Verifică dacă aplicarea mutărilor date pe starea curentă duce la soluție"""
        return reduce(lambda s, m: s.apply_move_inplace(m), moves, self.clone()) == self.solved()

    def clone(self):
        return NPuzzle(self.r, self.moves)
    
    def __str__(self) -> str:
        return str(self.N-1) + "-puzzle:" + str(self.r)
    def __repr__(self) -> str: 
        return str(self)
    def __eq__(self, other):
        return self.r == other.r
    def __lt__(self, other):
        return True
    def __hash__(self):
        return hash(tuple(self.r))
    
    
    def create_state(self, current_node):
        
        index = 0
        state = [[0 for col in range(self.side)] for row in range(self.side)]
        
        for i in range(self.side):
            for j in range(self.side):
                state[i][j] = current_node[index]
                index += 1
                
        return state
    
    
    def euclidean_distance(self, current_node, end_node):
        
        distance = 0
        final_state = self.create_state(end_node)
        current_state = self.create_state(current_node)
        
        for i in range(self.side):
            for j in range(self.side):
                for k in range(self.side):
                    for l in range(self.side):
                        if current_state[i][j] == final_state[k][l] and current_state[i][j] != self.BLANK:
                            distance += math.sqrt(pow((i - k), 2) + pow((j - l), 2))
                            
        return distance
    
    
    def manhattan_distance(self, current_node, end_node):
        
        distance = 0
        final_state = self.create_state(end_node)
        current_state = self.create_state(current_node)
        
        for i in range(self.side):
            for j in range(self.side):
                for k in range(self.side):
                    for l in range(self.side):
                        if current_state[i][j] == final_state[k][l] and current_state[i][j] != self.BLANK:
                            distance += abs(i - k) + abs(j - l)
                            
        return distance
    
    
    def get_possible_moves(self, current_node):
        
        possible_states = []
        current_state = self.create_state(current_node.r)
        next_state = deepcopy(current_state)
        next_node = deepcopy(current_node)
        
        for i in range(self.side):
            for j in range(self.side):
                if current_state[i][j] == self.BLANK:
                    if i != 0:
                        next_move = current_state[i-1][j]
                        next_state[i-1][j] = self.BLANK
                        next_state[i][j] = next_move
                        new_state = [next_state[i][j] for i in range(self.side) for j in range(self.side)]
                        next_node.r = new_state
                        possible_states.append(next_node)
                        next_state = deepcopy(current_state)
                        next_node = deepcopy(current_node)
                        
                    if i != self.side-1:
                        next_move = current_state[i+1][j]
                        next_state[i+1][j] = self.BLANK
                        next_state[i][j] = next_move
                        new_state = [next_state[i][j] for i in range(self.side) for j in range(self.side)]
                        next_node.r = new_state
                        possible_states.append(next_node)
                        next_state = deepcopy(current_state)
                        next_node = deepcopy(current_node)
                        
                    if j != 0:
                        next_move = current_state[i][j-1]
                        next_state[i][j-1] = self.BLANK
                        next_state[i][j] = next_move
                        new_state = [next_state[i][j] for i in range(self.side) for j in range(self.side)]
                        next_node.r = new_state
                        possible_states.append(next_node)
                        next_state = deepcopy(current_state)
                        next_node = deepcopy(current_node)
                        
                    if j != self.side-1:
                        next_move = current_state[i][j+1]
                        next_state[i][j+1] = self.BLANK
                        next_state[i][j] = next_move
                        new_state = [next_state[i][j] for i in range(self.side) for j in range(self.side)]
                        next_node.r = new_state
                        possible_states.append(next_node)
                        
                    return possible_states
                
                
    def best_heuristic(self, successors):
        
        best_heuristic = successors[0]
        for length in range(1, len(successors)):
            if best_heuristic[2] > successors[length][2]:
                best_heuristic = successors[length]
                
        return best_heuristic
    
    def get_path(self, visited):
        
        path = []
        start_node = self
        current_node = self.solved()
        path.insert(0, current_node)
        
        while current_node.r != start_node.r:
            current_node = visited[current_node]
            path.insert(0, current_node)
        
        return path
    
    
    def astar(self, h, limit):
        
        frontier = []
        start_node = self
        end_node = self.solved()
        discovered = {start_node: (None, 0)}
        heappush(frontier, (0 + h(start_node.r, end_node.r), start_node))

        while frontier:
            (current_cost, current_node) = heappop(frontier)
            
            if current_node == end_node and len(discovered) < limit:
                return "SUCCESS", len(discovered), discovered[current_node][1]
            elif len(discovered) >= limit:
                return "FAILURE"
                
            next_nodes = self.get_possible_moves(current_node)

            for node in next_nodes:
                new_cost = discovered[current_node][1] + 1
                if discovered.get(node) is None or new_cost < discovered[node][1]:
                    heappush(frontier, (new_cost + h(node.r, end_node.r), node))
                    discovered[node] = (current_node, new_cost)
                    
                    
    def beam_search(self, h, B, limit):
        
        start_node = self
        end_node = self.solved()
        beam = {start_node: None}
        visited = {start_node: None}
        
        while len(beam) != 0 and len(visited) < limit:
            successors = []
            selected = []
            
            for parent_node in beam:
                next_nodes = self.get_possible_moves(parent_node)
                successors += [{'state': (current_node, parent_node), 'heuristic': h(current_node.r, end_node.r)} for current_node in next_nodes]
            
            for next_state in successors:
                if next_state['state'][0] == end_node:
                    visited[next_state['state'][0]] = parent_node
                    return "SUCCESS", len(visited), len(self.get_path(visited))

            successors = sorted(successors, key=itemgetter('heuristic'))
            
            for breadth in range(B):
                if breadth < len(successors) and successors[breadth]['state'][0] not in visited:
                    selected.append(successors[breadth])

            beam = {}
            for item in selected:
                visited[item['state'][0]] = item['state'][1]
                beam[item['state'][0]] = item['state'][1]
                                
        return "FAILURE"
    
    
    def GLDS(self, h, limit):
        
        start_node = self
        visited = {start_node: None}
        discrepancy = 0
        
        while True:
            if self.iteration_GLDS(start_node, discrepancy, h, visited, limit) == "SUCCESS":
                return "SUCCESS", len(visited)
            
            discrepancy += 1
            
            
    def iteration_GLDS(self, state, discrepancy, h, visited, limit):
        
        successors = []
        end_node = self.solved()
        next_nodes = self.get_possible_moves(state)
        
        for current_node in next_nodes:
            if current_node == end_node:
                    return "SUCCESS"
            
            if current_node not in visited:
                successors += [(current_node, state, h(current_node.r, end_node.r))]
                
        if len(successors) == 0:
            return "FAILURE"
        
        if len(visited) > limit:
            return "FAILURE"
        
        best_h = self.best_heuristic(successors)
                
        if discrepancy == 0:
            visited[best_h[0]] = best_h[1]
            return self.iteration_GLDS(best_h[0], 0, h, visited, limit)
        
        else:
            
            successors.remove(best_h)
            
            while len(successors) != 0:
                
                next_h = self.best_heuristic(successors)
                successors.remove(next_h)
                visited[next_h[0]] = next_h[1]
                
                if self.iteration_GLDS(next_h[0], discrepancy - 1, h, visited, limit) == "SUCCESS":
                    return "SUCCESS"
                
            visited[best_h[0]] = best_h[1]
            
            return self.iteration_GLDS(best_h[0], discrepancy, h, visited, limit)
                
        
    def BLDS(self, h, B, limit):
        
        start_node = self
        end_node = self.solved()
        visited = {start_node: None}
        discrepancy = 0
        
        while True:
            if self.iteration_BLDS([{'state': (start_node, None), 'heuristic': h(start_node.r, end_node.r)}], discrepancy, B, h, visited, limit) == "SUCCESS":
                return "SUCCESS", len(visited)

            discrepancy += 1
            
            
    def iteration_BLDS(self, level, discrepancy, B, h, visited, limit):
        
        successors = []
        end_node = self.solved()
        
        for next_state in level:
            next_nodes = self.get_possible_moves(next_state['state'][0])
            
            for current_node in next_nodes:
                if current_node == end_node:
                    return "SUCCESS"
                
                if current_node not in visited:
                    successors += [{'state': (current_node, next_state['state'][0]), 'heuristic': h(current_node.r, end_node.r)}]
            
        if len(successors) == 0:
            return "FAILURE"
            
        if len(visited) + min([B, len(successors)]) > limit:
            return "FAILURE"
            
        successors = sorted(successors, key=itemgetter('heuristic'))
            
        if discrepancy == 0:
            next_level = []
            for breadth in range(B):
                if breadth < len(successors) and successors[breadth]['state'][0] not in visited:
                    next_level.append(successors[breadth])
                        
            for item in next_level:
                visited[item['state'][0]] = item['state'][1]
                    
            return self.iteration_BLDS(next_level, 0, B, h, visited, limit)
            
        else:
                
            next_level = []
            already_explored = B

            while len(successors) > already_explored:
                n = min([len(successors) - already_explored, B])

                for length in range(already_explored, already_explored + n + 1):
                    if length < len(successors) and successors[length]['state'][0] not in visited:
                        next_level.append(successors[length])

                for item in next_level:
                    visited[item['state'][0]] = item['state'][1]

                result = self.iteration_BLDS(next_level, discrepancy - 1, B, h, visited, limit)

                if result == "SUCCESS":
                    return "SUCCESS"

                already_explored += len(next_level)

            for breadth in range(B):
                if breadth < len(successors) and successors[breadth]['state'][0] not in visited:
                    next_level.append(successors[breadth])

            for item in next_level:
                visited[item['state'][0]] = item['state'][1]

            return self.iteration_BLDS(next_level, discrepancy, B, h, visited, limit)
                
            
# MLIMIT = 3 * 10 ** 9 # 2 GB RAM limit
# setrlimit(RLIMIT_DATA, (MLIMIT, MLIMIT))

# parametrii fiecarui joc

B = 100
limit = 100000
N = [4, 5, 6]

for game_size in N:
    
    # citirea din fisier a jocurilor easy
    
    f = open("files/problems"+str(game_size)+"-easy.txt", "r")
    input = f.readlines()
    f.close()
    problems = [NPuzzle.read_from_line(line) for line in input]

    for length in range(len(problems)):

        print("Game " + str(length+1))

        start_time = time.time()
        print("A*\n", NPuzzle.astar(problems[length], problems[length].manhattan_distance, limit))
        print(time.time() - start_time, "\n")

        start_time = time.time()
        print("Beam Search\n", NPuzzle.beam_search(problems[length], problems[length].manhattan_distance, B, limit))
        print(time.time() - start_time, "\n")

        start_time = time.time()
        print("GLDS\n", NPuzzle.GLDS(problems[length], problems[length].manhattan_distance, limit))
        print(time.time() - start_time, "\n")

        start_time = time.time()
        print("BLDS\n", NPuzzle.BLDS(problems[length], problems[length].manhattan_distance, B, limit))
        print(time.time() - start_time, "\n")
        
    # citirea din fisier a jocurilor hard
    
    f = open("files/problems"+str(game_size)+".txt", "r")
    input = f.readlines()
    f.close()
    problems = [NPuzzle.read_from_line(line) for line in input]

    for length in range(len(problems)):

        print("Game " + str(length+1))

        start_time = time.time()
        print("Beam Search\n", NPuzzle.beam_search(problems[length], problems[length].manhattan_distance, B, limit))
        print(time.time() - start_time, "\n")

        start_time = time.time()
        print("GLDS\n", NPuzzle.GLDS(problems[length], problems[length].manhattan_distance, limit))
        print(time.time() - start_time, "\n")

        start_time = time.time()
        print("BLDS\n", NPuzzle.BLDS(problems[length], problems[length].manhattan_distance, B, limit))
        print(time.time() - start_time, "\n")

# generare

# def genOne(side, difficulty):
#     state = NPuzzle(list(range(side * side))[1:] + [NPuzzle.BLANK])
#     for i in range(side ** difficulty + random.choice(range(side ** (difficulty//2)))):
#         s = state.apply_move(random.choice(NPuzzle.ACTIONS))
#         if s is not None: state = s
#     state.clear_moves()
#     return state

# print("Generare:")
# random.seed(4242)
# p = genOne(5, 4)
# p.display()

# problemele easy au fost generate cu dificultatile 4, 3, respectiv 2 (pentru marimile 4, 5, 6)
# celelalte probleme au toate dificultate 6


Game 1
Beam Search
 ('SUCCESS', 5464, 96)
2.7290446758270264 

